## ML Project

In this assignment we will be engaging with the stock price data for several hundred stocks over 5 years (./Data/prices-split-adjusted.csv.zip).  The data is from [Kaggle](https://www.kaggle.com/dgawlik/nyse#prices-split-adjusted.csv) and is available in its original form there under a CC0 license. (It's 50MB, but compressed it's only 15)

In this notebook we will be looking at financial forecasting with machine learning.  This is inherently one of the hardest problems in machine learning, because some of the most advanced and well-funded technical teams in the world are trying to use machine learning and other techniques to find patterns in the financial data.  When they find patterns and if they trade on those findings, prices will move in a way that makes those patterns less pronounced over time. This is not to say that this is not a fun and rewarding area.  Just do not get discouraged if you don't find an instant money machine.  

### Outline:
0. Background

1. Preparing our tools

2. Importing and describing the data.

3. Exploring, cleaning and visualizing the data

3. Developing analytics

4. Preparing and splitting our data

5. Building our first model

6. Extending to other ML models

7. Ideas for further strategies

8. Wrapping up

### Options
As we progress you are encouraged to take this dataset further. You are also encouraged to explore any aspects of the data. Develop your own algorithms. Be explicit about your inquiry and success in predicting effects on our world.

### Warning: Not financial advice
This exercise is meant purely for educational purposes, uses many simplifications and is not intended, nor should be considered as financial advice. There are many risks involved in implementation of financial trading strategies that are not considered nor described here.

### Setting up
If you have not yet set up your environment, you can easily do so with VS Code, and the python extension and Anaconda.  

For VSCode go here: [https://code.visualstudio.com/]

and then you can follow these instructions:
[https://code.visualstudio.com/docs/python/data-science-tutorial]

0. Background

Machine learning is of increasing importance in finance. As volumes of data grow ever faster, the need for machine driven models to find patterns in that data becomes ever more important.  In the ever-accelerating race to better process data into predictions about securities prices, machine learning has become an important tool, because it is good at finding patterns in large amounts of data.  Today we will be examining patterns in stock prices themselves to practice developing models to predict future set prices. If there are systematic trends, patterns or reversals, then we may detect them. 

While the chances that we discover totally new and unexploited price patterns today is low, we will practice organizing our data, creating and analyzing machine learning models that will give us the tools to develop state of the art signals of value.

Goals: 

1. Become familiar and practice the process of building machine learning models as they relate to financial data.  

2. Understand the special processing that is required when working with time series data such those found in finance.



### 1. Preparing our tools.

Let's review our standard imports:
- numpy for rapid numerical calculations with fast vectorized C implementations
- pandas for processing data
- matplotlib and Seaborn for visualizing charts
- scikit-learn (imported as sklearn) is the de facto standard machine learning library in the pydata ecosystem.  
 
Additionally, we will be using [pandas_profiling](https://github.com/pandas-profiling/pandas-profiling) which is a newer convenience package that helps by putting together much of our initial *boilerplate* exploratory data analysis code. 


In [ ]:
pip install pandas-profiling[notebook]

In [1]:
# Bring our tools in:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pandas_profiling import ProfileReport
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import TimeSeriesSplit

%matplotlib inline

### Importing and describing the data

Now we are ready to import our data.  The value rows can be set if you only want to import a small subset due to computer memory or speed constraints.

In [2]:
# We are using information from the data source description to know that date is a column containing just what is says
rows = 851264
file_path = 'data/prices-split-adjusted.csv'
stocks = pd.read_csv(file_path, nrows=rows, parse_dates=['date'], index_col='date')

Now that we have our data successfully loaded, let's explore what we have. First, summarize the dataframe via the info method to validate the data reading and parsing.  When looking at the info report, it is best practice to note that each column is the expected type, noting that strings are reported as object.  Also note if there are null values, how many values there are and what the columns are.  

We do not have a data dictionary in this case.  But if you were lucky enough to have access to a data dictionary, this is a good time to check that the dictionary matches what you actually have.  Discrepancies could be the result of mis-parsing, undocumented schema changes, documentation that is not up to date, or a number of other reasons.  

In [3]:
stocks.info() # Look at the descriptions of the columns

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 851264 entries, 2016-01-05 to 2016-12-30
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   symbol  851264 non-null  object 
 1   open    851264 non-null  float64
 2   close   851264 non-null  float64
 3   low     851264 non-null  float64
 4   high    851264 non-null  float64
 5   volume  851264 non-null  float64
dtypes: float64(5), object(1)
memory usage: 42.2+ MB


Everything looks as expected in info.  The string column symbol is the trading symbol, also known in finance as the ticker.  The dates were parsed as expected and all the other columns are numeric.  Now we can look at the first few rows of data to get a sample view.  

In [4]:
stocks.head(10)

,symbol,open,close,low,high,volume
date,,,,,,
2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0
2016-01-12,WLTW,115.510002,115.550003,114.500000,116.059998,1098000.0
2016-01-13,WLTW,116.459999,112.849998,112.589996,117.070000,949600.0
2016-01-14,WLTW,113.510002,114.379997,110.050003,115.029999,785300.0
2016-01-15,WLTW,113.330002,112.529999,111.919998,114.879997,1093700.0


While the head gives a good preview of a piece of the data, it may not be a great overall view of the entire dataset, especially for larger data sets or ones that may have been sorted at some point.  However, we are more comfortable that the dates were parsed correctly.  We can investigate numerical columns with describe.  

In [5]:
stocks.describe()

,open,close,low,high,volume
count,851264.000000,851264.000000,851264.000000,851264.000000,8.512640e+05
mean,64.993618,65.011913,64.336541,65.639748,5.415113e+06
std,75.203893,75.201216,74.459518,75.906861,1.249468e+07
min,1.660000,1.590000,1.500000,1.810000,0.000000e+00
25%,31.270000,31.292776,30.940001,31.620001,1.221500e+06
50%,48.459999,48.480000,47.970001,48.959999,2.476250e+06
75%,75.120003,75.139999,74.400002,75.849998,5.222500e+06
max,1584.439941,1578.130005,1549.939941,1600.930054,8.596434e+08



#### Pandas Profiling
Data exploration can start with a turnkey tool like pandas-profiling.  The important this with this is to make sure you actually look at the report and digest the output.  Make it the start of your investigation.

In [8]:
# Minimal avoids expensive calculations that won't have much insight for us and are slow.
profile = ProfileReport(stocks, minimal=True)
profile.to_widgets()


#### Exercise: 

Examine this report and see what insights you notice.  What do you notice about the data?  Are there ways to slice the data that would give more information?    

In [54]:
# Blank cell left for student exploration

# --- stocks in this dataset
#stocks.symbol.describe
#stocks['symbol'].value_counts(normalize=True)
#with pd.option_context('display.max_rows', 501):
#    stocks.symbol.value_counts()
stocks.symbol.unique()
#profile = ProfileReport(stocks, title='Pandas Profiling Report - Stocks', explorative=True)
#profile.to_widgets()

array(['WLTW', 'A', 'AAL', 'AAP', 'AAPL', 'ABC', 'ABT', 'ACN', 'ADBE',
       'ADI', 'ADM', 'ADP', 'ADS', 'ADSK', 'AEE', 'AEP', 'AES', 'AET',
       'AFL', 'AGN', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALK', 'ALL',
       'ALXN', 'AMAT', 'AME', 'AMG', 'AMGN', 'AMP', 'AMT', 'AMZN', 'AN',
       'ANTM', 'AON', 'APA', 'APC', 'APD', 'APH', 'ARNC', 'ATVI', 'AVB',
       'AVGO', 'AVY', 'AWK', 'AXP', 'AYI', 'AZO', 'BA', 'BAC', 'BAX',
       'BBBY', 'BBT', 'BBY', 'BCR', 'BDX', 'BEN', 'BHI', 'BIIB', 'BK',
       'BLK', 'BLL', 'BMY', 'BSX', 'BWA', 'BXP', 'C', 'CA', 'CAG', 'CAH',
       'CAT', 'CB', 'CBG', 'CBS', 'CCI', 'CCL', 'CELG', 'CERN', 'CF',
       'CHD', 'CHK', 'CHRW', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA',
       'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COH',
       'COL', 'COO', 'COP', 'COST', 'CPB', 'CRM', 'CSCO', 'CSX', 'CTAS',
       'CTL', 'CTSH', 'CTXS', 'CVS', 'CVX', 'CXO', 'D', 'DAL', 'DD', 'DE',
       'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 

Looking at an individual stock gives a much clearer impression of the distributions of each column.  Even if you can not do this for every stock, taking a sample can be very helpful.  


### 2. Exploring, cleaning and visualizing the data

#### Modeling our data
One of the most important steps in preparing your data is to think of how we want to think about it.  For those who have worked with SQL, this is like identifying what the key to your table is. For this data exploration, we can think of the index into the data as being a a compound key of both the symbol (the ticker) and the date.  We will process the data in a way that will make a time series model for each stock.  Predicting the future based on what has happened in the past for that individual stock.  

Thought Question: Can you think of other ways that you might want to consider this data?  What questions might prompt you to think of this data having a different data model?  The data model is not fixed but is a lens that lets us look at our data.  




#### Visualizing our data
It is often helpful to look at our data visually to see if there are any issues that "look funny."  With experience, just looking at the data can help us understand it in a short amount of time.  This is often the step where domain expertise (in this case the financial markets) is especially useful.  Be sure to look at the histograms of the report above to see if they make sense to you.  

Exercise: 
Are there other visualizations that would give you greater understanding?

In [ ]:
pip install bokeh

In [41]:
import requests
import datetime
import math
from bokeh.plotting import figure, show, output_file

In [11]:
# keep stocks as is
df = stocks

# change datatype 
df.open = df.open.astype(float)
df.close = df.close.astype(float)
df.high = df.high.astype(float)
df.low = df.low.astype(float)
df.volume = df.volume.astype(int)

df.reset_index(inplace=True)

# change datetime
df['date'] = pd.to_datetime(df['date'])

# sort according to date
df = df.sort_values(by=['date'])

In [13]:
df.head()

,date,symbol,open,close,low,high,volume
646,2010-01-04,SYMC,18.040001,18.400000,18.01,18.530001,8322300
473,2010-01-04,IFF,41.509998,42.009998,41.50,42.020000,286000
474,2010-01-04,ILMN,31.120001,30.549999,30.42,31.230000,1793700
475,2010-01-04,INTC,20.790001,20.879999,20.73,21.030001,47800900
476,2010-01-04,INTU,31.090000,30.809999,30.77,31.150000,2353000


In [14]:
#df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 851264 entries, 646 to 851263
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    851264 non-null  datetime64[ns]
 1   symbol  851264 non-null  object        
 2   open    851264 non-null  float64       
 3   close   851264 non-null  float64       
 4   low     851264 non-null  float64       
 5   high    851264 non-null  float64       
 6   volume  851264 non-null  int32         
dtypes: datetime64[ns](1), float64(4), int32(1), object(1)
memory usage: 45.5+ MB


In [15]:
inc = df.close > df.open # instances where stock value increased 
dec = df.open > df.close # instances where stock value decreased 
w = 12*60*60*1000 # half day in ms

In [37]:
# --- stock 1 --- 
stk1 = 'WLTW'
df_wltw = df[df.symbol == stk1] 

In [38]:
df_wltw.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600


In [43]:
# specify tools 
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

#stock_name = stk1
title = stk1 + ' Chart'

p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, title = title)

p.xaxis.major_label_orientation = math.pi/4
p.grid.grid_line_alpha=0.3

p.segment(df_wltw.date, df_wltw.high, df_wltw.date, df_wltw.low, color="black")
p.vbar(df_wltw.date[inc], w, df_wltw.open[inc], df_wltw.close[inc], fill_color="#D5E1DD", line_color="black")
p.vbar(df_wltw.date[dec], w, df_wltw.open[dec], df_wltw.close[dec], fill_color="#F2583E", line_color="black")


GlyphRenderer(id='2931', ...)

In [44]:
#Store as a HTML file
#output_file("stock_information.html", title="candlestick.py example")
# Display in browser
show(p)

 add a second stock

In [46]:
# which stock?
stk2 = 'ILMN'
df_ILMN = df[df.symbol == stk2] 
title = stk2 + ' Chart'


In [47]:
df_ILMN.head()

,date,symbol,open,close,low,high,volume
474,2010-01-04,ILMN,31.120001,30.549999,30.420000,31.230000,1793700
942,2010-01-05,ILMN,30.370001,30.350000,29.760000,30.500000,3039700
1410,2010-01-06,ILMN,30.240000,32.220001,30.240000,32.430000,4065700
1878,2010-01-07,ILMN,32.240002,32.770000,32.000000,33.209999,9060000
2346,2010-01-08,ILMN,32.599998,33.150002,32.080002,33.279999,2681600


In [48]:

p.segment(df_ILMN.date, df_ILMN.high, df_ILMN.date, df_ILMN.low, color="red")
p.vbar(df_ILMN.date[inc], w, df_ILMN.open[inc], df_ILMN.close[inc], fill_color="#D5E1DD", line_color="red")
p.vbar(df_ILMN.date[dec], w, df_ILMN.open[dec], df_ILMN.close[dec], fill_color="#F2583E", line_color="red")


GlyphRenderer(id='3243', ...)

In [51]:
show(p)

Observations:
Second stock (ILMN) has been around for way longer than the first stock (WLTW), yet it can be seen that when WLTW first started, ILMN was doing better than it is doing by the end of 2016. 
In the period of 1/2060 till 1/2017:
ILMN's value has been decreasing whilst WLTW's is slowly increasing. 

### 3. Feature engineering
For modelling data with machine learning, it is helpful to transform the data into a form that is closer to the theoretical expectations where the ML models should perform well. Let's transform the data into returns and generate other features.  We will transform returns with logarithms based on financial research that log returns are closer to normally distributed and (statistically) stable. 

The function below is just a sample of feature transformations.  Taking the logarithms can help deal with skewed data as we saw we have in the pandas-profile report.

To be honest, which variables you use and how you transform them is largely dependent on domain expertise and traditions of the field. It can also be a matter of trial and error, although that can lead to overfitting.  We will discuss overfitting a little bit later.

In [52]:
def feature_target_generation(df):
    """
    df: a pandas dataframe containing numerical columns
    num_days_ahead: an integer that can be used to shift the prediction value from the future into a prior row.
    """

    # The following line ensures the data is in date order    
    features = pd.DataFrame(index=df.index).sort_index() 
    features['f01'] = np.log(df.close / df.open) # intra-day log return
    features['f02'] = np.log(df.open / df.close.shift(1)) # overnight log return

    features['f03'] = df.volume # try both regular and log volume
    features['f04'] = np.log(df.volume) 
    features['f05'] = df.volume.diff() # 1-day absolute change in volume
    features['f06'] = df.volume.pct_change() # 1-day relative change in volume
    # The following are rolling averages of different periods
    features['f07'] = df.volume.rolling(5, min_periods=1).mean().apply(np.log)
    features['f08'] = df.volume.rolling(10, min_periods=1).mean().apply(np.log)
    features['f09'] = df.volume.rolling(30, min_periods=1).mean().apply(np.log)

    # More of our original data: low, high and close
    features['f10'] = df.low 
    features['f11'] = df.high
    features['f12'] = df.close
    # The Intraday trading spread measures how far apart the high and low are
    features['f13'] = df.high - df.low 
    # These are log returns over different time periods 
    features['f14'] = np.log(df.close / df.close.shift(1)) # 1 day log return
    features['f15'] = np.log(df.close / df.close.shift(5)) # 5 day log return
    features['f16'] = np.log(df.close / df.close.shift(10)) # 10 day log return
    return features

In Machine Learning, we need to predict something, typically called our target or predictor. 

Let's predict the value of the stock price 10 days into the future, using "prediction_horizon".  Here 10 is a hyperparameter that is somewhat arbitrary.  We may want to try different horizons to see if we are better at predicting the near future or the long term.  

The ticker lets us start by testing on a single stock for speed in training.  

We want to look at different periods of history to see how we do.  We will use overlapping sets with n_splits.  

In [53]:
# Let's generate a list of tickers so we can easily select them
ticker_list = stocks.symbol.unique()

# these are hyperparameters you can play with or tune
prediction_horizon = -5 # this is a negative number by convention
ticker = 'MSFT' # choose any ticker
n_splits = 5 

# Make an individual model for each ticker/symbol
features = feature_target_generation(stocks[stocks.symbol==ticker])

### 5. Preparing and splitting our data
It is important that we separate our training and test data.  Since our rows are already time ordered, we can easily do splits.  This is one of the areas where times series data is different from other machine learning problems.  

In [55]:
# We are trying to predict the price prediction_horizon days in the future.  
# So we take the future value and move it prediction_horizon into the past 
# to line up our data in the Scikit-learn format.  

y = features.f12.shift(prediction_horizon)  # f12 is the original 'close'

# The latest (prediction_horizon) rows will have nans because we have no future data, so let's drop them.
shifted = ~np.isnan(y)

X = features[y.notna()] # Remove the rows that do not have valid target values
y = y[shifted] # Remove the rows that do not have valid target values

# Split the history into different backtesting regimes
tscv = TimeSeriesSplit(n_splits=n_splits)

print(tscv)

TimeSeriesSplit(max_train_size=None, n_splits=5)


In [56]:
 # Review the features
 features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1762 entries, 544 to 851076
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   f01     1762 non-null   float64
 1   f02     1761 non-null   float64
 2   f03     1762 non-null   int32  
 3   f04     1762 non-null   float64
 4   f05     1761 non-null   float64
 5   f06     1761 non-null   float64
 6   f07     1762 non-null   float64
 7   f08     1762 non-null   float64
 8   f09     1762 non-null   float64
 9   f10     1762 non-null   float64
 10  f11     1762 non-null   float64
 11  f12     1762 non-null   float64
 12  f13     1762 non-null   float64
 13  f14     1761 non-null   float64
 14  f15     1757 non-null   float64
 15  f16     1752 non-null   float64
dtypes: float64(15), int32(1)
memory usage: 227.1 KB


### 6. Building our first model
This is a regression problem.  Why is that so? 




#### Linear regression
In our ML framework we can use linear regression, just as in standard statistics or econometrics.

In [57]:
def model_ts_report(model, tscv, X, y, impute=False):
    """
    Fit the model and then run time series backtests.
    """
    # Loop through the backtests
    for train_ind, test_ind in tscv.split(X): 
        # Report on the time periods
        print(f'Train is from {X.iloc[train_ind].index.min()} to {X.iloc[train_ind].index.max()}. ')
        print(f'Test is from {X.iloc[test_ind].index.min()} to {X.iloc[test_ind].index.max()}. ')
        # Generate training and testing features and target for each fold.
        X_train, X_test = X.iloc[train_ind], X.iloc[test_ind]
        y_train, y_test = y.iloc[train_ind], y.iloc[test_ind]

        if impute==True:
            # Since linear regression cannot deal with NaN, we need to impute.  There may be the better choices.
            X_train.fillna(0, inplace=True)
            X_test.fillna(0, inplace=True)
        
        # Fit the model
        model.fit(X_train, y_train)

        # Predict and measure on the training data
        y_pred_train = model.predict(X_train) 
        print("Training results:")
        print("RMSE:", mean_squared_error(y_train, y_pred_train, squared=False))

        # Predict and measure on the testing data
        y_pred_test = model.predict(X_test)
        print("Test results:")
        print("RMSE:", mean_squared_error(y_test, y_pred_test, squared=False))
        print("")
    

In [59]:
from sklearn.linear_model import LinearRegression
# Fit and report on a linear model
lm = LinearRegression()
model_ts_report(lm, tscv, X, y, impute=True)

Train is from 544 to 139365. 
Test is from 139835 to 276605. 
Training results:
RMSE: 0.7646691835730663
Test results:
RMSE: 0.8845462997580511

Train is from 544 to 276605. 
Test is from 277075 to 415644. 
Training results:
RMSE: 0.804692104280704
Test results:
RMSE: 0.9584479421164054

Train is from 544 to 415644. 
Test is from 416132 to 558601. 
Training results:
RMSE: 0.8388520878620891
Test results:
RMSE: 1.2199724751308518

Train is from 544 to 558601. 

c:\users\abeli\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
c:\users\abeli\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
c:\users\abeli\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu


Test is from 559091 to 702761. 
Training results:
RMSE: 0.913565597444975
Test results:
RMSE: 1.8578363718516142

Train is from 544 to 702761. 
Test is from 703259 to 848576. 
Training results:
RMSE: 1.1573573014110943
Test results:
RMSE: 1.5623450048029444



c:\users\abeli\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
c:\users\abeli\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


As we look at our results, we can see that in each period we do better in training than in testing.  That is typical of finance and machine learning more generally. 
Interestingly, we are able to explain 90% of the variance with our first linear model.  

If you have questions about the root-mean-squared-error (RMSE), please see the Microsoft learn module on machine learning.

#### Ensemble Model
Let's try a Random Forest, which is a commonly used model that blends a group of decision trees, each of which have access to a sub-sample of features.  It is commonly used because it it tends to work well with relatively little tuning of hyperparameters and is somewhat less likely to overfit.  This is NOT a classic model commonly used in econometrics, mainly because it does not have a linear structure that corresponds with econometric theory.  However, it is often used in predictions for items other than finance.  


In [60]:
# Initiate a Random Forest
rf = RandomForestRegressor()
model_ts_report(rf, tscv, X, y, impute=True) # Report on the random forest

Train is from 544 to 139365. 
Test is from 139835 to 276605. 


c:\users\abeli\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


Training results:
RMSE: 0.23433399182267542
Test results:
RMSE: 1.1557848765641885

Train is from 544 to 276605. 
Test is from 277075 to 415644. 


c:\users\abeli\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


Training results:
RMSE: 0.24736887384776862
Test results:
RMSE: 1.3639759072190314

Train is from 544 to 415644. 
Test is from 416132 to 558601. 


...

Training results:
RMSE: 0.2555382490085588
Test results:
RMSE: 5.328385248744586

Train is from 544 to 558601. 
Test is from 559091 to 702761. 


c:\users\abeli\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


Training results:
RMSE: 0.2763046958625175
Test results:
RMSE: 2.905831648397354

Train is from 544 to 702761. 
Test is from 703259 to 848576. 


c:\users\abeli\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


Training results:
RMSE: 0.38174298836332105
Test results:
RMSE: 5.338487967059192



By using a Random Forest, we are able to bring down our training data error metrics by a lot (over 50% decrease on RMSE)!  However, our test results are not as good for most of the time slices.  This is an indication that we are *overfitting* our model to the training data.  This model would likely not do as well in production as it would in our backtests.  Why?  Because our models have not done well on any of the new data, outside the time period during which they were trained. 

### Open Ended Exercise

Now is your turn to go ahead and improve these models.  Some areas that might help could be to: 
- Tune the existing models (Random forest has a number of parameters that may help) 
- Clean the existing data (Fill missing values better) 
- Try other models such as Support Vector Regressor, Extra Trees Regressor or ElasticNet 
- Try this for more stocks (Just becasue it did not work for one stock, it may still be useful for most stocks) 
- Get more features, through transformations or outside data  


  